In [1]:
%reload_ext autotime
import pandas as pd
import requests
import geopandas as gpd
from tqdm.auto import tqdm
import time
import os

In [2]:
poly = gpd.read_file("polygons.geojson")
poly = poly[poly.id.str.startswith("nzd")]
poly.set_index("id", inplace=True)
poly

,area,id_sorted,northing,geometry
id,,,,
nzd0001,3.069093e+06,nzd0001,-4.085604e+06,"POLYGON ((172.96406 -34.43054, 172.99324 -34.4..."
nzd0002,1.507931e+06,nzd0002,-4.086100e+06,"POLYGON ((172.93456 -34.42857, 172.95856 -34.4..."
nzd0003,2.484730e+06,nzd0003,-4.087232e+06,"POLYGON ((173.0098 -34.42055, 173.00088 -34.44..."
nzd0006,9.619509e+05,nzd0004,-4.090732e+06,"POLYGON ((173.00532 -34.46694, 173.00612 -34.4..."
nzd0007,3.109433e+06,nzd0005,-4.095516e+06,"POLYGON ((172.99847 -34.48102, 173.00081 -34.5..."
...,...,...,...,...
nzd0317,9.986110e+05,nzd0556,-4.971830e+06,"POLYGON ((172.32302 -40.7187, 172.3121 -40.728..."
nzd0316,4.300058e+06,nzd0557,-4.966165e+06,"POLYGON ((172.37076 -40.70229, 172.39571 -40.6..."
nzd0313,2.291974e+07,nzd0558,-4.939929e+06,"POLYGON ((172.89281 -40.50518, 172.73306 -40.5..."


In [3]:
sitename = "nzd0001"
dates = pd.to_datetime(pd.read_csv(f"data/{sitename}/transect_time_series.csv").dates).dt.round("10min")
display(dates)
point = poly.geometry[sitename].centroid
point.x, point.y

0     1999-08-17 22:10:00+00:00
1     1999-10-04 22:10:00+00:00
2     2000-04-29 22:10:00+00:00
3     2000-06-16 22:10:00+00:00
4     2000-08-03 22:10:00+00:00
                 ...           
155   2024-04-23 22:20:00+00:00
156   2024-05-09 22:20:00+00:00
157   2024-05-25 22:20:00+00:00
158   2024-06-26 22:20:00+00:00
159   2024-07-12 22:20:00+00:00
Name: dates, Length: 160, dtype: datetime64[ns, UTC]

(172.97690966086745, -34.42150701355086)

In [4]:
def get_tide_for_dt(point, datetime):
    while True:
        r = requests.get("https://api.niwa.co.nz/tides/data", params={
            "lat": point.y,
            "long": point.x,
            "numberOfDays": 2,
            "startDate": str(datetime.date()),
            "datum": "MSL",
            "interval": 10, # 10 minute resolution
            "apikey": os.environ["NIWA_API_KEY"]
        }, timeout=(30,30))
        if r.status_code == 200:
            df = pd.DataFrame(r.json()["values"])
            df.index = pd.to_datetime(df.time)
            return df.value[datetime]
        elif r.status_code == 429:
            sleep_seconds = 30
            # sleep for x seconds to refresh the count
            print(f'Num of API reqs exceeded, Sleeping for: {sleep_seconds} seconds...')
            time.sleep(sleep_seconds)

results = []
for date in tqdm(dates):
    result = get_tide_for_dt(point, date)
    results.append({
        "dates": date,
        "tide": result
    })

  0%|          | 0/160 [00:00<?, ?it/s]

In [5]:
df = pd.DataFrame(results)
df = df.rename(columns={"date": "dates"})
df.set_index("dates", inplace=True)
df.to_csv(f"data/{sitename}/tides.csv")
df

,tide
dates,
1999-08-17 22:10:00+00:00,0.16
1999-10-04 22:10:00+00:00,-0.83
2000-04-29 22:10:00+00:00,-0.60
2000-06-16 22:10:00+00:00,0.24
2000-08-03 22:10:00+00:00,0.94
...,...
2024-04-23 22:20:00+00:00,0.29
2024-05-09 22:20:00+00:00,0.78
2024-05-25 22:20:00+00:00,0.67


In [6]:
FES = pd.read_csv(f"shoreline_data_run6/New_Zealand/{sitename}/tide_levels_fes2014.csv")
FES.index = pd.to_datetime(FES.dates).dt.round("10min")
FES["tide levels"]

dates
1999-08-17 22:10:00+00:00    0.131615
1999-10-04 22:10:00+00:00   -0.764331
2000-02-09 22:10:00+00:00    0.831439
2000-04-29 22:10:00+00:00   -0.573744
2000-05-15 22:10:00+00:00   -0.302252
                               ...   
2021-05-01 22:20:00+00:00    0.573917
2021-05-17 22:20:00+00:00    0.306780
2021-06-18 22:20:00+00:00   -0.458240
2021-07-04 22:20:00+00:00   -0.668631
2021-09-22 22:20:00+00:00    0.789146
Name: tide levels, Length: 182, dtype: float64

In [7]:
(df.tide - FES["tide levels"]).describe()

count    135.000000
mean      -0.003214
std        0.067335
min       -0.159995
25%       -0.045775
50%        0.007195
75%        0.054883
max        0.112977
dtype: float64